## Вариант 3. Контроль количества параметров

**Цель:** Создание компактной сети.

- Создайте архитектуру, используя **не более 50 000 параметров**.
- **Условие:** Один слой должен быть **1x1 сверткой**, чтобы уменьшить число каналов.
- **Эксперимент:** Подсчитайте количество параметров каждого слоя и убедитесь, что общая сумма не превышает лимита.

In [50]:
import torch
from torch import nn
import torch.nn.functional as F

In [60]:
class CompConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_parameters_num = sum(p.numel() for p in self.conv1.parameters())

        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 16, kernel_size=1)
        self.conv2_parameters_num = sum(p.numel() for p in self.conv2.parameters())

        self.fc1 = nn.Linear(16 * 8 * 8, 32)
        self.fc1_parameters_num = sum(p.numel() for p in self.fc1.parameters())

        self.fc2 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.fc2_parameters_num = sum(p.numel() for p in self.fc2.parameters())

    def count_parameters(model):
        return sum(p.numel() for p in model.parameters())

    def forward(self, x):
        # Первый сверточный слой
        x = self.conv1(x) # (64x32x32)
        x = self.relu(x)
        x = self.pool(x) # (64x16x16)
        
        # Второй сверточный слой
        x = self.conv2(x) # (16x16x16)
        x = self.relu(x)
        x = self.pool(x) # (16x8x8)

        # Выходной слой
        x = x.view(x.size(0), -1) # (1x1x32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # (1x1x10)
        return x

model = CompConvNet()

test_tensor = torch.randn(1, 3, 32 ,32)
model(test_tensor)


tensor([[ 0.0248,  0.0382, -0.1677,  0.0964, -0.1512, -0.2534,  0.1326,  0.1499,
          0.0283,  0.0644]], grad_fn=<AddmmBackward0>)

In [52]:
model.count_parameters()

35962

In [53]:
print((3*3*3 + 1) * 64 + (1*1*64 + 1) * 16 + (16*8*8 + 1) * 32 + (32 + 1) * 10) # Рассчетное количество параметров
print(model.conv1_parameters_num + model.conv2_parameters_num + model.fc1_parameters_num + model.fc2_parameters_num) # Действительное количество параметров

35962
35962
